In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 24
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.8e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2800000.00, reward:-343.91
 Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2800000.00, reward:-343.91
 Time: 0 | new_e:79.4, cum_e:79.4, new_i:70.9, cum_i:70.9
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495699.7, E: 221.7, A: 0.0, I: 70.9, D: 0.0, R: 7.6, Budget: 2800000.00, reward:-831.52
 Time: 1 | S: 25495697.0, E: 215.0, A: 0.0, I: 70.9, D: 0.0, R: 6.0, Budget: 2800000.00, reward:-831.52
 Time: 1 | new_e:111.4, cum_e:190.8, new_i:98.1, cum_i:169.0
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.

[0.15602486041513297, 0.15745945396698147]
[0.8120872439756422, 0.8135096099289049]
[0.09799834382996749, 0.09827709630210713]
[0.04116805668679551, 0.041818538229688895]
 Time: 9 | S: 25350496.5, E: 642.8, A: 1660.3, I: 2478.5, D: 3.9, R: 140718.1, Budget: 2704582.17, reward:-15648.11
 Time: 9 | S: 25350832.0, E: 601.0, A: 1660.3, I: 2478.5, D: 3.9, R: 140690.0, Budget: 2704582.17, reward:-15648.11
 Time: 9 | new_e:301.1, cum_e:3552.1, new_i:447.5, cum_i:3398.4
[[9.9999976e-01 9.9999964e-01 1.3809672e-08 9.9999976e-01 8.3943773e-03
  9.9999976e-01 9.9999940e-01 9.9999940e-01 1.5259381e-07 7.4614655e-12
  9.9999964e-01 9.9999940e-01 9.9999988e-01 9.9999964e-01 9.9999964e-01
  9.9999940e-01 9.9999976e-01 9.9999964e-01 9.9999964e-01 8.9521073e-03
  9.9999976e-01 9.5551330e-01 9.9999976e-01 1.0000000e+00]
 [9.9998736e-01 9.9999952e-01 9.9999940e-01 9.9999940e-01 9.9999964e-01
  9.9999988e-01 9.9999940e-01 9.9999952e-01 9.9999952e-01 9.9999976e-01
  9.9999976e-01 9.9999952e-01 9.9999607e-0

[0.15642173518166105, 0.15645603616518297]
[0.8134545292332374, 0.8134625976207727]
[0.09825250964392002, 0.09825260325901586]
[0.041802387096982974, 0.041802626350060344]
 Time: 13 | S: 25084833.2, E: 219.8, A: 1502.6, I: 2635.8, D: 5.7, R: 406803.0, Budget: 2526957.27, reward:-18023.35
 Time: 13 | S: 25085175.0, E: 228.0, A: 1502.6, I: 2635.8, D: 5.7, R: 406749.0, Budget: 2526957.27, reward:-18023.35
 Time: 13 | new_e:101.8, cum_e:4073.6, new_i:252.4, cum_i:4597.6
[[1.0000000e+00 1.0000000e+00 2.6113083e-13 1.0000000e+00 3.3544097e-03
  1.0000000e+00 1.0000000e+00 1.0000000e+00 2.7170863e-10 6.0777067e-13
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9994743e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999237e-01 1.000000

[0.15643360758035846, 0.15643507218393243]
[0.8134616577234979, 0.8134616734684198]
[0.09825259356538026, 0.09825259365798286]
[0.0418026136226281, 0.04180261380679093]
 Time: 17 | S: 24820957.0, E: 122.7, A: 1093.3, I: 2377.4, D: 7.7, R: 671442.0, Budget: 2353085.89, reward:-19728.73
 Time: 17 | S: 24821231.0, E: 134.0, A: 1093.3, I: 2377.4, D: 7.7, R: 671371.0, Budget: 2353085.89, reward:-19728.73
 Time: 17 | new_e:56.3, cum_e:4439.3, new_i:167.8, cum_i:5431.6
[[1.0000000e+00 1.0000000e+00 9.3007405e-13 1.0000000e+00 2.8097692e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.2125960e-11 7.0493091e-13
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9998915e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999964e-01 1.0000000e+0

[0.1564341015213313, 0.15643414437303718]
[0.8134616716574626, 0.8134616717339962]
[0.09825259364838669, 0.09825259364848381]
[0.041802613796987964, 0.041802613797154095]
 Time: 21 | S: 24562957.1, E: 73.1, A: 677.8, I: 1874.1, D: 9.1, R: 930408.8, Budget: 2182444.26, reward:-20401.35
 Time: 21 | S: 24563175.0, E: 86.0, A: 677.8, I: 1874.1, D: 9.1, R: 930332.0, Budget: 2182444.26, reward:-20401.35
 Time: 21 | new_e:33.2, cum_e:4578.3, new_i:102.6, cum_i:5914.8
[[1.0000000e+00 1.0000000e+00 3.2507355e-10 1.0000000e+00 2.0524482e-03
  1.0000000e+00 1.0000000e+00 1.0000000e+00 2.9765421e-13 5.1337107e-10
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9995947e-01
  1.0000000e+00 9.9999988e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9999988e-01 

...

 Time: 73 | S: 21961107.7, E: 0.0, A: 0.2, I: 23.9, D: 13.0, R: 3534855.2, Budget: 30850.74, reward:-21070.11
 Time: 73 | S: 21961242.0, E: 0.0, A: 0.2, I: 23.9, D: 13.0, R: 3534813.0, Budget: 30850.74, reward:-21070.11
 Time: 73 | new_e:2.0, cum_e:4682.4, new_i:1.4, cum_i:6525.8
[[1.0000000e+00 1.0000000e+00 5.7668575e-10 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.8760599e-14 2.4521135e-14
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9996197e-01
  1.1825393e-14 9.9997282e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9966455e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 3.9026876e-08 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.5274234e-06 9.9995220e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000

 Time: 77 | S: 22038983.0, E: 12.7, A: 0.1, I: 35.4, D: 13.1, R: 3456955.8, Budget: 30850.74, reward:-21168.76
 Time: 77 | S: 22039118.0, E: 4.0, A: 0.1, I: 35.4, D: 13.1, R: 3456915.0, Budget: 30850.74, reward:-21168.76
 Time: 77 | new_e:8.1, cum_e:4703.4, new_i:6.7, cum_i:6542.8
[[1.0000000e+00 1.0000000e+00 2.4876551e-08 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 6.9014742e-17 9.4185585e-14
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999976e-01
  1.1551288e-13 9.9999988e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 8.6338041e-06 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.5397145e-02 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.000000

 Time: 81 | S: 22115074.2, E: 53.3, A: 0.1, I: 82.5, D: 13.3, R: 3380776.7, Budget: 30850.74, reward:-21513.34
 Time: 81 | S: 22115265.0, E: 8.0, A: 0.1, I: 82.5, D: 13.3, R: 3380739.0, Budget: 30850.74, reward:-21513.34
 Time: 81 | new_e:28.6, cum_e:4779.3, new_i:24.8, cum_i:6608.1
[[1.0000000e+00 1.0000000e+00 2.8487437e-09 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 2.3107027e-14 1.7653802e-13
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  2.4566986e-16 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 5.1215365e-09 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9964988e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000

 Time: 85 | S: 22189256.9, E: 190.4, A: 0.0, I: 250.1, D: 13.9, R: 3306288.7, Budget: 30850.74, reward:-22700.22
 Time: 85 | S: 22189653.0, E: 58.0, A: 0.0, I: 250.1, D: 13.9, R: 3306240.0, Budget: 30850.74, reward:-22700.22
 Time: 85 | new_e:98.8, cum_e:5042.7, new_i:86.3, cum_i:6837.8
[[1.0000000e+00 1.0000000e+00 1.0114419e-12 1.0000000e+00 9.6781421e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 7.1807463e-11 1.5206988e-16
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.7079569e-03
  8.7842917e-13 9.9899954e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999976e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9999964e-01 4.5581029e-08 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 3.2794465e-09 9.9999714e-01 1.0000000e+00]
 [1.0000000e+00 1.

 Time: 89 | S: 22261008.3, E: 660.8, A: 0.0, I: 831.3, D: 15.7, R: 3233483.9, Budget: 30850.74, reward:-26788.82
 Time: 89 | S: 22261912.0, E: 341.0, A: 0.0, I: 831.3, D: 15.7, R: 3233398.0, Budget: 30850.74, reward:-26788.82
 Time: 89 | new_e:341.1, cum_e:5952.2, new_i:298.2, cum_i:7632.8
[[1.0000000e+00 1.0000000e+00 8.9712266e-05 1.0000000e+00 5.8520341e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 2.1591886e-13 4.5300344e-10
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9997246e-01
  1.1686270e-13 9.9999714e-01 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  9.9999988e-01 9.9999988e-01 1.0000000e+00 8.9781423e-07 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9999976e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 6.0359966e-03 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00

In [6]:
(time.time()-tic) / ( T - start_time + 1)

74.74192549481111